In [1]:
import pandas as pd
import metadata_parser as mp
import requests
from bs4 import BeautifulSoup as bs
import re

In [2]:
# Call dataset
springer_open = pd.read_excel("./paper-links/PaperLinks_SpringerOpen.xlsx")

In [ ]:
# Get the paper metadata
list_year, list_title, list_conf_jour, list_authors, list_keywords, list_abstract = [], [], [], [], [], []

for paper_url in springer_open["Paper Links"]:
    # Get page
    page = mp.MetadataParser(paper_url)
    page_bs = requests.get(paper_url)
    parsed_bs = bs(page_bs.text,'html.parser') 
    # Get Authors
    authors_list = page.get_metadatas('citation_author')
    try:
        for i in range(0,len(authors_list)):
            if i == 0:
                authors = authors_list[i].split(",", 1)[1]+" "+authors_list[i].split(",", 1)[0]
            else:
                authors = authors+","+authors_list[i].split(",", 1)[1]+" "+authors_list[i].split(",", 1)[0]
            if i == len(authors_list)-1:
                authors = authors.strip()
                break
        # Get Year
        year = page.get_metadatas('citation_publication_date')
        year = year[0].split("/", 1)[0]
        # Get Title
        title = page.get_metadatas('citation_title')
        title = title[0]
        # Get Conf/Journal Name
        conf_jour_name = page.get_metadatas('citation_journal_title')
        conf_jour_name = conf_jour_name[0]
        # Get Keywords
        keywords_list = parsed_bs.find_all(class_='c-article-subject-list__subject')
        for i in range(0,len(keywords_list)):
            if i == 0:
                keywords = re.search('<span>(.*)</span>', str(keywords_list[i])).group(1)
            else:
                keywords = keywords+", "+re.search('<span>(.*)</span>', str(keywords_list[i])).group(1)
            if i == len(keywords_list)-1:
                keywords = keywords.strip()
                break
        # Get Abstract
        abstract = page.get_metadatas('description')
        abstract = abstract[0]

        # Append to the list
        list_year.append(year)
        list_title.append(title)
        list_conf_jour.append(conf_jour_name)
        list_authors.append(authors)
        list_keywords.append(keywords)
        list_abstract.append(abstract)
        
    except:
        # Append to the list
        list_year.append("not_a_paper")
        list_title.append("not_a_paper")
        list_conf_jour.append("not_a_paper")
        list_authors.append("not_a_paper")
        list_keywords.append("not_a_paper")
        list_abstract.append("not_a_paper")       

In [4]:
# Concatenate list as new column
springer_open["Year"], springer_open["Title"], springer_open["Conf/Journal Name"] = list_year, list_title, list_conf_jour
springer_open["Authors"], springer_open["Keywords"], springer_open["Abstract"] = list_authors, list_keywords, list_abstract

In [5]:
springer_open.to_excel("./paper-metadata/PaperMetadata_Unclean_SpringerOpen.xlsx", index = False)